In [14]:
import os
import myhelper
from dotenv import load_dotenv
import pyupbit
from datetime import datetime as dt
from datetime import timedelta
import myutil as util
from dataclasses import dataclass
import yaml

load_dotenv()

access_key = os.getenv("UPBIT_OPEN_API_ACCESS_KEY")
secret_key = os.getenv("UPBIT_OPEN_API_SECRET_KEY")
upbit = pyupbit.Upbit(access_key, secret_key)
my_balance = upbit.get_balances()


In [12]:
# my_balance

order_history = [{'uuid': '11697def7-5ada-405f-b0f3-053801d5b190',
  'side': 'ask',
  'ord_type': 'market',
  'price': None,
  'state': 'done',
  'market': 'KRW-LTC',
  'created_at': '2021-03-21T14:43:40+09:00',
  'volume': '0.07336815',
  'remaining_volume': '0.0',
  'reserved_fee': '0.0',
  'remaining_fee': '0.0',
  'paid_fee': '8.39331636',
  'locked': '0.0',
  'executed_volume': '0.07336815',
  'trades_count': 1}]

len(order_history)

1

In [13]:
buy('KRW-GLM', 10000)

저장한다.


{'11697def7-5ada-405f-b0f3-053801d5b190': 'bid'}

In [28]:
def convert_to_ticker(coin):
    return f"KRW-{coin}"

def get_my_coins(balances:bool = False):
    '''
    - get_my_coins() : 보유중인 코인만을 'KRW-aaa'의 형태로 돌려준다.(list)
    - get_my_coins(True) : 보유중인 코인과 그 소유량을 돌려준다.(locked 제외, dict)
    '''
    my_balance = upbit.get_balances()

    if balances:
        bals = {}
        tickers = get_my_coins()
        for ticker in tickers:
            balance = upbit.get_balance(ticker)
            bals[ticker] = balance
        return bals
    else:
        return [f"KRW-{x['currency']}" for x in my_balance]

def sell(arg):
    '''
    - sell(ticker) : 해당 코인 전량을 매도한다. 
    - sell(uuid) : 해당 UUID분량만 매도한다. 
    '''
    if len(arg) > 15:
        ticker = arg
        volume = upbit.get_balance(arg)
        ask = upbit.sell_market_order(ticker, volume)
    else:
        order = upbit.get_order(arg)
        volume = order['volume']
        ticker = order['market']
        ask = upbit.sell_market_order(ticker, volume)
        res = {}
        res[arg] = ask[0]['price']
        util.save_yml('data/order_history.yml', **res)

    return ask['uuid']

def limit_sell(arg, price):
    '''
    - limit_sell(ticker) : 해당 코인 전량을 매도한다. 
    - limit_sell(uuid) : 해당 UUID분량만 매도한다. 
    '''   
    if len(arg) > 15:
        ticker = arg
        volume = upbit.get_balance(arg)
    else:
        order = upbit.get_order(arg)
        volume = order['volume']
        ticker = order['market']

    ask_price = pyupbit.get_tick_size(price)
    ask = upbit.sell_limit_order(ticker, ask_price, volume)

    res = {}
    res[arg] = ask[0]['price']
    util.save_yml('data/order_history.yml', **res)

    return ask['uuid']

def buy(ticker, money:float):
    volume = money/pyupbit.get_current_price(ticker)
    bid = upbit.buy_market_order(ticker, volume)

    res = {}
    res[bid[0]['uuid']] = bid[0]['price']
    util.save_yml('data/order_history.yml', **res)
    
    return bid['uuid']

def limit_buy(ticker, target_price:float):
    money = 5100
    volume = money/target_price
    bid_price = pyupbit.get_tick_size(target_price)
    bid = upbit.buy_limit_order(ticker, bid_price, volume)
    
    res = {}    
    res[bid[0]['uuid']] = bid[0]['price']
    util.save_yml('data/order_history.yml', **res)   
    
    return bid['uuid']
    
def cancel_order(uuid:str):
    return upbit.cancel_order(uuid)

def get_history(info=False):
    '''
    현재 소유중인 코인에 기초하여, 
    '남아 있는' 모든 예약주문을 돌려준다.
    (가지고 있지 않은 코인은 판매할 수 없으므로. 구매예약은 고려하지 않음.)

    - info=False : uuid만 반환한다.(list)
    - info=True : 모든 정보를 반환한다.(dict)
    '''
    ods = {}
    _, *tickers = get_my_coins()

    for ticker in tickers:
        ods[ticker] = upbit.get_order(ticker)

    if info:
        return ods
    else:
        return [v[0]['uuid'] for k,v in ods.items()]
    


In [23]:
upbit.get_balances()

[{'currency': 'KRW',
  'balance': '5209.8792455',
  'locked': '0',
  'avg_buy_price': '0',
  'avg_buy_price_modified': True,
  'unit_currency': 'KRW'},
 {'currency': 'ASTR',
  'balance': '0',
  'locked': '41.82194616',
  'avg_buy_price': '241.5',
  'avg_buy_price_modified': False,
  'unit_currency': 'KRW'}]

In [26]:
def main():
    with open('data/order_history.yml', 'r') as f:
        order_history = yaml.safe_load(f)

    for order in order_history:
        order

main()

{'currency': 'ASTR', 'balance': '0', 'locked': '41.82194616', 'avg_buy_price': '241.5', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}


In [ ]:
class Order:
    

In [21]:
{'uuid': '22ca2984-5e02-4d53-8d56-1420fe1fad7f',
 'side': 'ask',
 'ord_type': 'limit',
 'price': '243',
 'state': 'done',
 'market': 'KRW-ASTR',
 'created_at': '2024-03-15T09:05:03+09:00',
 'volume': '62.28348341',
 'remaining_volume': '0',
 'reserved_fee': '0',
 'remaining_fee': '0',
 'paid_fee': '7.6141558468725',
 'locked': '0',
 'executed_volume': '62.28348341',
 'trades_count': 1,
}.keys()

dict_keys(['uuid', 'side', 'ord_type', 'price', 'state', 'market', 'created_at', 'volume', 'remaining_volume', 'reserved_fee', 'remaining_fee', 'paid_fee', 'locked', 'executed_volume', 'trades_count'])

In [5]:


def sell(upbit, my_balance) -> list:
    my_coin:list = []

    try:
        # 보유코인 관리
        for i in range(len(my_balance)):
            ticker = f"KRW-{my_balance[i]['currency']}"
            avg_buy_price = float(my_balance[i]['avg_buy_price'])
            balance = float(my_balance[i]['balance'])
            cut_line = avg_buy_price* (1.0 - 0.015)
            earn_line = avg_buy_price* (1.0 + 0.025)    

            # 구매가가 있다면 ( == 현금잔고가 아님 )
            if avg_buy_price:
                current_price = pyupbit.get_current_price(ticker)
                if current_price < cut_line or current_price > earn_line:
                    # 
                    rate = myutil.moving_rate(avg_buy_price,current_price)
                    print(f'{ticker} 매각. 수익률 : {rate:,.2f}')
                else:
                    my_coin.append(ticker)
            else:
                cash = balance
                locked = float(my_balance[i]['locked'])
                print(f'보유현금 : {cash:,.2f}, 예약중 : {locked:,.2f}')

    except Exception as e:
        print('Exception cut_loss():', e)

    return my_coin

# sell(upbit, my_balance)

# cash = my_balance[0]['balance']


ModuleNotFoundError: No module named 'myorders'

In [ ]:
@dataclass
class Coin:
    currency: str
    balance: float
    locked: float
    avg_buy_price: float
    avg_buy_price_modified: bool
    unit_currency: str

    def ticker(self):
        ticker = "-",join(self.unit_currency,self.currency)


In [44]:
import pandas as pd
csv_file = 'sample_order.csv'
def update_df(data, file_path=None):
    
    df = pd.DataFrame(data)

    if not file_path:
        file_path = 'sample_order.csv'
    else:
        file_path = file_path

    if os.path.exists(file_path):
        _df = pd.read_csv(file_path)
        df = pd.concat([df, _df], axis=0).drop_duplicates()
    
    df.to_csv(file_path, index=False, encoding='utf-8')

data = [{'uuid': '999999999-9b4f-4bb0-9c03-30318e3ff10a',
  'side': 'ask',
  'ord_type': 'limit',
  'price': '250000.0',
  'state': 'wait',
  'market': 'KRW-LTC',
  'created_at': '2021-03-25T14:10:53+09:00',
  'volume': '1.0',
  'remaining_volume': '1.0',
  'reserved_fee': '0.0',
  'remaining_fee': '0.0',
  'paid_fee': '0.0',
  'locked': '1.0',
  'executed_volume': '0.0',
  'trades_count': 0}]

update_df(data)
df = pd.read_csv('sample_order.csv')
df.drop(index=1)
df.index.names = ['Genie']
df.set_index = 'uuid'

ValueError: If using all scalar values, you must pass an index